# Challenge: timerand

In [1]:
import requests
from datetime import datetime, timezone
import base64
from hashlib import md5
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.asymmetric import rsa, padding as asymmetric_padding
from cryptography.hazmat.primitives import serialization, hashes, padding as symmetric_padding
from cryptography.hazmat.backends import default_backend  
def get_challnge(challenge:str) -> str:
    server = "https://ciberseguridad.diplomatura.unc.edu.ar/cripto"
    email = "alvarmaciel@gmail.com"
    response =  requests.get(f"{server}/{challenge}/{email}/challenge")
    return response.text
    
def parse_challenge(raw: str):
    lines = raw.splitlines()
    # encontrar la línea en blanco que separa encabezado y cuerpo
    sep = lines.index('')
    headers = lines[:sep]
    body_lines = lines[sep+1:]
    # unir todas las líneas del cuerpo
    b64_concat = ''.join(body_lines)
    blob = base64.b64decode(b64_concat)
    return headers, blob

In [2]:
raw_challenge = get_challnge("timerand")
raw_challenge

'From: User <user@example.com>\nDate: Fri Jan 14 19:45:58 UTC 2022\nTo: alvarmaciel@gmail.com\n\nJKjGe49INxspqtMMtm6hesPCxag4ljpDzLhlmyagYkEpcuzz7cSdMYMaigA1Rqkwqk+KtVxODwNV8vGM2tUsWgTJ4xA4RFrx0Ww2o/o27ocLKUwfQiVPENw198AI25iAljOAmOioNJd/IRmzLzEW88RknUjRF4udo+0YlNeOtwGWuUR4iMj7+otL1XnymNFMxkXzq+PeKloTS9SbmzKDVQr8HG9BjE5Fd/emVJQ+y9K/qDU637IfIhOGjeKX/k03f9xMt00UWTi852J2M+aF1xyOAmKTADSzlEDU1jWfD1Czy8TFD0VJnnImtSjOpThocCjwnJTKxQhjFaiPJjqPAb/jSNhpkWNFebPUrXeRDOAIKYGegZfekvITvyU0RFziAJEVUv1LmUEOyoKzlFCv1OYEAJNn70qNzYvhBHE1L+IrubFzCl4o7xeiJlN2sIVdCaaGENfV3gMgbGPtAtlbOczHWN9Gz1Sq4oFBdNWgF58Z5Q8vkoMKG5o0WQJqduohZG5KrDp9M3jjSbvH8ZhoBg==\n'

In [3]:
headers, blob = parse_challenge(raw_challenge)
print(headers)
print(blob)

['From: User <user@example.com>', 'Date: Fri Jan 14 19:45:58 UTC 2022', 'To: alvarmaciel@gmail.com']
b'$\xa8\xc6{\x8fH7\x1b)\xaa\xd3\x0c\xb6n\xa1z\xc3\xc2\xc5\xa88\x96:C\xcc\xb8e\x9b&\xa0bA)r\xec\xf3\xed\xc4\x9d1\x83\x1a\x8a\x005F\xa90\xaaO\x8a\xb5\\N\x0f\x03U\xf2\xf1\x8c\xda\xd5,Z\x04\xc9\xe3\x108DZ\xf1\xd1l6\xa3\xfa6\xee\x87\x0b)L\x1fB%O\x10\xdc5\xf7\xc0\x08\xdb\x98\x80\x963\x80\x98\xe8\xa84\x97\x7f!\x19\xb3/1\x16\xf3\xc4d\x9dH\xd1\x17\x8b\x9d\xa3\xed\x18\x94\xd7\x8e\xb7\x01\x96\xb9Dx\x88\xc8\xfb\xfa\x8bK\xd5y\xf2\x98\xd1L\xc6E\xf3\xab\xe3\xde*Z\x13K\xd4\x9b\x9b2\x83U\n\xfc\x1coA\x8cNEw\xf7\xa6T\x94>\xcb\xd2\xbf\xa85:\xdf\xb2\x1f"\x13\x86\x8d\xe2\x97\xfeM7\x7f\xdcL\xb7M\x14Y8\xbc\xe7bv3\xe6\x85\xd7\x1c\x8e\x02b\x93\x004\xb3\x94@\xd4\xd65\x9f\x0fP\xb3\xcb\xc4\xc5\x0fEI\x9er&\xb5(\xce\xa58hp(\xf0\x9c\x94\xca\xc5\x08c\x15\xa8\x8f&:\x8f\x01\xbf\xe3H\xd8i\x91cEy\xb3\xd4\xadw\x91\x0c\xe0\x08)\x81\x9e\x81\x97\xde\x92\xf2\x13\xbf%4D\\\xe2\x00\x91\x15R\xfdK\x99A\x0e\xca\x82\xb3\x94P\xaf\xd4\x

In [7]:
def decipher(b64_decoded_message: bytes, timestamp: int) -> list[tuple[int, str]]:
    decipher_options = []

    rsa_encrypted_key = b64_decoded_message[:128]
    iv = b64_decoded_message[128:144]
    cipher_text = b64_decoded_message[144:]
    if len(iv) != 16 or len(cipher_text) % 16 != 0:
        raise ValueError("IV o ciphertext con longitud inválida")

    # start = int(timestamp) * 1_000_000
    # end   = start + 1_000_000  # no inclusivo
    base_us = int(timestamp) * 1_000_000
    printable = set(range(32, 127)) | {9, 10, 13}
    
    for t in range(1_000_000):
        try:
            t_us = base_us + t
            key_seed = t_us.to_bytes(8, "big")
            key = md5(key_seed).digest()

            dec = Cipher(algorithms.AES(key), modes.CBC(iv)).decryptor()
            padded = dec.update(cipher_text) + dec.finalize()
            unpad = symmetric_padding.PKCS7(128).unpadder()
            pt = unpad.update(padded) + unpad.finalize()

            head = pt[:20]
            # Evitar división por cero si el plaintext es muy corto
            if not head: 
                continue

            # Verificar si es texto imprimible
            if sum(b in printable for b in head) / len(head) >= 0.9:
                try:
                    txt = pt.decode("ascii")
                    print(f"ÉXITO! Encontrado en el microsegundo: {t}")
                    return [(t_us, txt)]
                except UnicodeDecodeError:
                    # Si es imprimible pero no es ASCII puro, podría ser válido pero lo ignoramos
                    continue
        except Exception:
            # Ahora el 'continue' solo se ejecuta si hay un error de padding u otro.
            continue

    return []
    
    

In [8]:
lines = raw_challenge.splitlines()
    # fecha UTC exacta desde el header
date_line = next(l for l in lines if l.startswith("Date: "))
print(date_line)
date_str = headers[1].replace("Date: ", "")
print(date_str)
base_sec = int(datetime.strptime(date_str, "%a %b %d %H:%M:%S UTC %Y").replace(tzinfo=timezone.utc).timestamp())
print(base_sec)

Date: Fri Jan 14 19:45:58 UTC 2022
Fri Jan 14 19:45:58 UTC 2022
1642189558


In [23]:
# date = int(datetime(2022,1,14,19,45,58, tzinfo=timezone.utc).timestamp()) 
descifrado = decipher(blob, base_sec)
descifrado[0][1]

ÉXITO! Encontrado en el microsegundo: 258342


'TeX is potentially the most significant invention in typesetting in this\ncentury.  It introduces a standard language for computer typography, and in\nterms of importance could rank near the introduction of the Gutenberg press.\n\t\t-- Gordon Bell'

In [22]:
server = "https://ciberseguridad.diplomatura.unc.edu.ar/cripto"
email = "alvarmaciel@gmail.com"
response = requests.post(
    f"{server}/timerand/{email}/answer",
    files = { "message": descifrado[0][1]}
)
print(f"Status code: {response.status_code}")
print(response.text)

Status code: 200
¡Ganaste!

